In [ ]:
import os
import pickle
import sparse
# import librosa
import mir_eval
import fluidsynth
import pretty_midi
import numpy as np
import IPython.display

from glob import glob
from utils.drum_utils import *
from utils.common_utils import *

%load_ext autoreload
%autoreload 2

### Preprocessing

In [ ]:
data = []
count = 0 # num of bars
data_path = '../drumVAE/data/drummer1'
save_path = 'midi_data.pkl'

In [ ]:
for dir_path in glob(data_path+'/session*'):
    for file_path in glob(dir_path+'/*.mid'):
        try:
            pm = pretty_midi.PrettyMIDI(file_path)

            # time signature 4/4 check
            if not check_time_sign(pm, num=4, denom=4): 
                continue

            start_time = pm.get_onsets()[0]
            beats = pm.get_beats(start_time) #beat location in second
            tempo = pm.estimate_tempo()#bpm
            fs = change_fs(beats)

            # for each inst
            for inst in pm.instruments:
        #         print(inst.notes)
                if inst.is_drum == True:
                    drum_roll = quantize_drum(inst, fs, start_time)
                    drum_roll = windowing(drum_roll) #64 / 16 = 4 measure
                    drum_roll = hot_encoding(drum_roll)

                    for i in range(0, drum_roll.shape[0]):
                        # to reduce size of data, sparse encoding
                        data.append(sparse.COO(drum_roll[i]))
        except:
            continue

        # verbose
        count += 1
        if count % 100 == 0:
            print('Files iterations %d' % count)

In [ ]:
len(data)

In [ ]:
# play music example
fs = 10
pm = drum_play(data[10].todense().astype('float32'), fs)
IPython.display.Audio(pm.fluidsynth(fs=16000), rate=16000)

In [ ]:
# save pickle
with open(save_path, 'wb') as f:
    print('File saved!')
    pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)

# laod pickle
with open(save_path, 'rb') as f:
    a = pickle.load(f)